# Sonification of f0 annotations
In this notebook, we illustrate the sonification of fundamental frequency annotations using the **libsoni.f0** module.

In [ ]:
import numpy as np
import pandas as pd
import os
import librosa
from IPython import display as ipd

from libsoni.core import f0
from libsoni.util.utils import mix_sonification_and_original, get_preset

Fs= 22050

## Simple scenario: C Major Triad
To start with a simple example, let's look at the fundamental frequencies of a **C major triad**.
<img src="data/demo_f0/C-Dur-DM.png" alt="C-Major-Triad" width="500" height="600">
The frequencies corresponding to the notes are:

| Note | Frequency (Hz) |
|------|----------------|
| C4   |     261.63     |
| E4   |     329.63     |
| G4   |     392.00     |
| C5   |     523.25     |

In [ ]:
# Define list of frequencies for C Major triad
C_Major_Triad_f0s = [261.63, 329.63, 392.00, 523.25, 0.0]

To sonify these notes, or more precisely their frequencies, we need an array of time positions at which the notes are to be played. Let's play the first note at 0.25 seconds for 0.5 seconds and all other notes consecutively for 0.5 seconds as well.

In [ ]:
# Define starting time position in seconds
start_sec = 0.25

# Define duration for each frequency
duration_note_sec = 0.5

# Create array from time information
time_positions = np.arange(start_sec, len(C_Major_Triad_f0s) * duration_note_sec, duration_note_sec)

The function **sonify_f0** from the module f0 takes a Nx2-dimensional numpy.ndarray containing the time positions in the first column and the f0s in the second column.

In [ ]:
# Create Nx2-dimensional ndarray time-f0
time_f0 = np.column_stack((time_positions, C_Major_Triad_f0s))

print('    Start   |     f0     ')
print('-------------------------')
for row in time_f0: print('    %.2f s  |   %.2f Hz ' % (row[0], row[1]))

### Sonified C Major Triad

In [ ]:
# Sonification using libsoni
sonified_C_Major_Triad_f0 = f0.sonify_f0(time_f0=time_f0, fs=Fs)

print('Sonified C Major triad:')
ipd.display(ipd.Audio(sonified_C_Major_Triad_f0, rate=Fs))

### Customizing the Sonification
To adjust the sonification, the function **sonify_f0** offers the possibility to set the so-called partial frequencies as well as their amplitudes to create a certain timbre. Let's say we want to use the fundamental frequency f0 as well as two times, three times and four times the fundamental frequency for sonification. For the amplitudes we want to set 1, 1/2, 1/3, 1/4.
| Custom frequencies | Custom amplitudes |
|--------------------|------------------|
| f0                 | 1                |
| 2*f0               | 1/2              |
| 3*f0               | 1/3              |
| 4*f0               | 1/4              |

In [ ]:
# Set custom_partials
custom_partials = np.array([1,2,3,4])

# Set amplitudes for custom_partials
custom_partials_amplitudes = np.array([1,1/2,1/3,1/4])

# Sonification with custom parital settings using libsoni
sonified_C_Major_Triad_f0 = f0.sonify_f0(time_f0=time_f0,
                                         partials=custom_partials,
                                         partials_amplitudes=custom_partials_amplitudes,
                                         fs=Fs)


print('Sonified C Major Triad with custom parials:')
ipd.display(ipd.Audio(sonified_C_Major_Triad_f0, rate=Fs))

## Scenario 1: *Ach Gott und Herr* by *J.S. Bach* (Bach10 Dataset [1])
"Ach Gott und Herr" is a sacred choral composition by the Baroque composer Johann Sebastian Bach. The piece typically features a four-part choir, supported by an instrumental ensemble. Bach often employed various instrumental combinations to enhance the choral texture and evoke different emotional layers. The following excerpt comprises a violin, a clarinet, a saxophone and a bassoon.

<img src="data/demo_f0/01-AchGottUndHerr_score.png" alt="Locus Iste" width="800" height="800">

In [ ]:
bach_audio, _ = librosa.load(os.path.join('data_audio','demo_f0','01-AchGottundHerr.wav'), sr=Fs)

print('"Ach Gott und Herr", by J. S. Bach:')
ipd.display(ipd.Audio(bach_audio, rate=Fs))

### Preparing the data
For the excerpt above, we load the corresponding .csv tables containing the time and f0 information for each instrument.

In [ ]:
# Load .csv-data for each instrument
violin_df = pd.read_csv(os.path.join('data_csv','demo_f0','01-AchGottundHerr_violin.csv'), sep=';')
clarinet_df = pd.read_csv(os.path.join('data_csv','demo_f0','01-AchGottundHerr_clarinet.csv'), sep=';')
saxophone_df = pd.read_csv(os.path.join('data_csv','demo_f0','01-AchGottundHerr_saxophone.csv'), sep=';')
bassoon_df = pd.read_csv(os.path.join('data_csv','demo_f0','01-AchGottundHerr_bassoon.csv'), sep=';')

print('Extract from violin_df:')
ipd.display(violin_df.iloc[100:105])

For the sonification of f0 annotations with presets we use the function **sonify_f0_with_presets** from **libsoni.f0**. This function takes a dictionary with presets as key with a Nx2-dimensional numpy.ndarray containing time and f0 information as value. 

In [ ]:
# Create dictionary
bach_dict = {'violin':violin_df.to_numpy(),
             'clarinet':clarinet_df.to_numpy(),
             'saxophone':saxophone_df.to_numpy(),
             'bassoon':bassoon_df.to_numpy()}

ipd.display(bach_dict)

### Working with presets
Since the above way of customizing sounds seems a bit unintuitive and more suitable for computational applications, the **libsoni.f0** module also offers the possibility to work with presets. In the following we illustrate this approach with our Bach example.
*(To view the different presets, see libsoni/utils/presets.json)*

In [ ]:
# get information about preset 
violin_preset = get_preset('violin')
ipd.display(violin_preset)

### Sonification

In [ ]:
bach_sonified = f0.sonify_f0_with_presets(bach_dict)

bach_sonified_w_original = mix_sonification_and_original(sonification=bach_sonified,
                                                         original_audio=bach_audio,
                                                         gain_lin_sonification=1.0,
                                                         gain_lin_original_audio=0.5)

print('Original audio:')
ipd.display(ipd.Audio(bach_audio, rate=Fs))

print('Sonified with libsoni:')
ipd.display(ipd.Audio(bach_sonified, rate=Fs))

print('Original audio with sonification (stereo):')
ipd.display(ipd.Audio(bach_sonified_w_original, rate=Fs))

## Scenario 2: 'SATB'  *Locus Iste* by *Anton Bruckner* (Dagstuhl Choirset [2])
"Locus Iste" is a sacred motet composed by Anton Bruckner, a renowned Austrian composer of the Romantic era. This composition, often performed in choral settings, showcases Bruckner's mastery of harmonies and expressive depth.
"SATB" is an abbreviation used to describe the voicing and arrangement of a choir in choral music. It stands for Soprano, Alto, Tenor, and Bass, representing the four main vocal ranges in a choir. When applied to "Locus Iste" by Anton Bruckner, which is a choral composition, SATB signifies how the voices are organized and distributed within the piece.

<img src="data/demo_f0/Locus_iste_score.png" alt="Locus Iste" width="800" height="800">

In [ ]:
satb_audio, _ = librosa.load(os.path.join('data_audio','demo_f0','DCS_LI_QuartetA_Take04_StereoReverb_STM.wav'), sr=Fs)

print('"Locus Iste" by Anton Bruckner:')
ipd.display(ipd.Audio(satb_audio, rate=Fs))

### Preparing the data
For the excerpt above, we load the corresponding .csv tables containing the time and f0 information for each voice.

In [ ]:
# Load .csv-data for each instrument
soprano_df = pd.read_csv(os.path.join('data_csv','demo_f0','DCS_LI_QuartetA_Take04_S2_LRX.csv'), sep=';')
alto_df = pd.read_csv(os.path.join('data_csv','demo_f0','DCS_LI_QuartetA_Take04_A1_LRX.csv'), sep=';')
tenor_df = pd.read_csv(os.path.join('data_csv','demo_f0','DCS_LI_QuartetA_Take04_T1_LRX.csv'), sep=';')
bass_df = pd.read_csv(os.path.join('data_csv','demo_f0','DCS_LI_QuartetA_Take04_B1_LRX.csv'), sep=';')

print('Extract from soprano_df:')
ipd.display(soprano_df.iloc[100:105])

As in the previous example, we again arrange the data as a dictionary. This time we use the presets **soprano**, **alto**, **tenor** and **bass**.

In [ ]:
satb_dict = {'soprano':soprano_df.to_numpy(),
             'alto':alto_df.to_numpy(),
             'tenor':tenor_df.to_numpy(),
             'bass':bass_df.to_numpy()}

satb_sonified = f0.sonify_f0_with_presets(satb_dict)
satb_sonified_w_original = mix_sonification_and_original(sonification=satb_sonified,
                                                         original_audio=satb_audio,
                                                         gain_lin_original_audio=0.05)

print('Original audio:')
ipd.display(ipd.Audio(satb_audio, rate=Fs))

print('Sonified with libsoni')
ipd.display(ipd.Audio(satb_sonified, rate=Fs))

print('Original audio with sonification (stereo)')
ipd.display(ipd.Audio(satb_sonified_w_original, rate=Fs))

## References

[1] Zhiyao Duan, Bryan Pardo and Changshui Zhang, “Multiple fundamental frequency estimation by modeling spectral peaks and non-peak regions,” IEEE Transactions of Audio Speech Language Process., vol. 18, no. 8, pp. 2121–2133, 2010.

[2] S. Rosenzweig, H. Cuesta, C. Weiß, F. Scherbaum, E. Gómez, and M. Müller, “Dagstuhl ChoirSet: A multitrack dataset for MIR research on choral singing,” Transactions of the International Society for Music Information Retrieval (TISMIR), vol. 3, no. 1, pp. 98–110, 2020.